# Data Generation for Demonstrating Multicollinearity

> Data simulated with the causal structure defined in @fig-causal. For use in [When is Multicollinearity an Issue?](/multicollinearity.html)

```{dot}
//| label: fig-causal
//| fig-cap: Causal Model of the data generating process
//| file: ../graphs/vifs/causal_graph.dot
```

In [ ]:
#| default_exp synthetic_data.multicollinearity_datagen

In [ ]:
#| hide
from nbdev.showdoc import *
import pprint
from itertools import product
import warnings
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
import pymc as pm
from tueplots import bundles
from tueplots.constants.color import rgb
import seaborn as sns

In [ ]:
#| hide
plt.rcParams.update(bundles.beamer_moml())
plt.rcParams.update({'figure.dpi': 300})
plt.rcParams['font.family'] = 'DeJavu Serif'
plt.rcParams['font.serif'] = ['Times New Roman']

In [ ]:
#| hide
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
#| export
#| hide
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import xarray as xr
from typing import Optional

In [ ]:
#| export
def hill(
    x, # input array
    K=1, # Half saturation point
    n=1.2 # Shape parameter
    ):
        return x**n/(K**n + x**n)

In [ ]:
#| export
def sample_random_data(
    N_weeks: int, # Number of weeks to generate
    include_hidden_confounds:bool=False, # Should hidden confounds be included in the dataset
    random_seed: int|None = None, # Random Seed
) -> xr.Dataset: # Dataset containing the variables described by the above causal model
    t = np.linspace(0, (N_weeks-1)/52., N_weeks)
    dates = pd.date_range("2021-01-01", periods=N_weeks, freq="W-MON")
    rng = np.random.default_rng(random_seed)

    ## Define Seasonal component
    seasonal_coeffs_sin = rng.normal(0, 2, 3)/np.arange(1, 4)**2
    seasonal_coeffs_cos = rng.normal(0, 2, 3)/np.arange(1, 4)**2
    season = (
        np.stack([np.sin(i*2*np.pi*t) for i in range(1, 4)]).T @ seasonal_coeffs_sin
        + np.stack([np.cos(i*2*np.pi*t) for i in range(1, 4)]).T @ seasonal_coeffs_cos
    )
    season = xr.DataArray((season-min(season))/(max(season)-min(season))*2-1, coords={"Period": dates}, dims="Period")

    ## Define Price Movements
    base_price = 3.50
    change_points = np.unique(np.clip(rng.exponential(4.0, size=int(N_weeks/4.0)).cumsum().astype(int), 0, N_weeks-1))
    changes = rng.normal(0, .1, size=len(change_points))
    price = np.zeros(N_weeks)
    price[change_points] = changes
    price = np.exp(price.cumsum())*base_price
    price = xr.DataArray(price, coords={"Period": dates}, dims="Period")

    ## Define Impressions
    social_impressions = rng.gamma(20, np.exp(6+season.values + rng.normal(0, .5, N_weeks)))
    social_impressions = xr.DataArray(social_impressions, coords={"Period": dates}, dims="Period")

    olv_impressions = rng.gamma(40, np.exp(6+.3*season.values + rng.normal(0, .7, N_weeks)))
    olv_impressions = xr.DataArray(olv_impressions, coords={"Period": dates}, dims="Period")

    ## Define Demand
    social_hill_params = {
        "K": rng.lognormal(0, .2),
        "n": rng.exponential(1) + 1
    }
    olv_hill_params = {
        "K": rng.lognormal(0, .2),
        "n": rng.exponential(1) + 1
    }

    
    
    olv_effect =  hill(olv_impressions/olv_impressions.where(olv_impressions>0).median(), **olv_hill_params)
    social_effect = hill(social_impressions/social_impressions.where(social_impressions>0).median(), **social_hill_params)

    social_beta = rng.lognormal(-2, .3)
    olv_beta = rng.lognormal(-2, .3)

    season_effect = rng.normal(0, .2)
    slope = rng.normal(0, .2)
    base_demand = np.log(1e2)
    price_elasticity = -rng.lognormal(0, .3)
    log_demand = (
        slope*np.linspace(0, 1, N_weeks) 
        + social_beta*social_effect 
        + olv_beta*olv_effect 
        + season_effect*season 
        + price_elasticity*np.log(price)
        + base_demand 
        + rng.normal(0, .05, size=N_weeks))
    demand = xr.DataArray(
        np.exp(log_demand),
        coords={"Period": dates},
        dims="Period"
    )

    # Search Query Data
    search_query = rng.gamma(10, np.exp(np.log(demand)*2 + 0.1*season + 6 + rng.normal(0, .2, size=N_weeks)))
    search_query = xr.DataArray(
        search_query,
        coords={"Period": dates},
        dims="Period"
    )

    # Auction
    auction = rng.beta(10, 100, size=N_weeks)
    auction = xr.DataArray(
        auction,
        coords={"Period": dates},
        dims="Period"
    )

    # Search Impressions
    paid_search_impressions = search_query*auction

    # Search Clicks
    ctr_logodds = (
        -8
        + rng.normal(0, .1, size=N_weeks) 
        + 2 * demand/demand.mean()
        + 1.0/((search_query/search_query.where(search_query>0).median())**1.5 + .8**1.5)
        )

    ctr = 1/(1+np.exp(-ctr_logodds))
    paid_search_clicks = ctr * paid_search_impressions

    # Organic Search
    organic_search = rng.beta(90, 10, size=N_weeks) * search_query
    
    
    #Sales
    sales = np.exp(
        .1*np.log(organic_search) 
        + .4 * np.log(paid_search_clicks)
        + 1 * np.log(demand)
        + np.log(price)
        + rng.normal(0, .05, size=N_weeks)
        )

    if include_hidden_confounds:
        dataset = xr.Dataset({
            "price": price, 
            "season":season,
            'social_impressions': social_impressions,
            "olv_impressions": olv_impressions,
            "demand": demand,
            "search_query": search_query,
            "auction": auction,
            "paid_search_impressions": paid_search_impressions,
            "paid_search_clicks": paid_search_clicks,
            "organic_search": organic_search,
            'sales': sales
        }).assign_attrs(olv_params=olv_hill_params, social_params=social_hill_params)
        return dataset

    dataset = xr.Dataset({
            "price": price, 
            #"season":season,
            'social_impressions': social_impressions,
            "olv_impressions": olv_impressions,
            #"demand": demand,
            "search_query": search_query,
            #"auction": auction,
            "paid_search_impressions": paid_search_impressions,
            "paid_search_clicks": paid_search_clicks,
            "organic_search": organic_search,
            'sales': sales
        }).assign_attrs(olv_params=olv_hill_params, social_params=social_hill_params)
    return dataset

In [ ]:
dataset = sample_random_data(156, random_seed=2, include_hidden_confounds=True)
dataset.head()

<xarray.Dataset> Size: 480B
Dimensions:                  (Period: 5)
Coordinates:
  * Period                   (Period) datetime64[ns] 40B 2021-01-04 ... 2021-...
Data variables:
    price                    (Period) float64 40B 3.856 3.767 3.767 4.3 4.3
    season                   (Period) float64 40B -0.9613 -0.9732 ... -1.0
    social_impressions       (Period) float64 40B 3.972e+03 ... 1.385e+03
    olv_impressions          (Period) float64 40B 7.405e+03 ... 4.911e+03
    demand                   (Period) float64 40B 35.99 38.55 39.04 32.23 28.31
    search_query             (Period) float64 40B 3.612e+06 ... 1.636e+06
    auction                  (Period) float64 40B 0.1327 0.1269 ... 0.08248
    paid_search_impressions  (Period) float64 40B 4.794e+05 ... 1.349e+05
    paid_search_clicks       (Period) float64 40B 1.974e+03 2.884e+03 ... 552.6
    organic_search           (Period) float64 40B 3.154e+06 ... 1.432e+06
    sales                    (Period) float64 40B 1.373e+04 ... 6.214e+03
Attributes:
    olv_params:     {'K': 0.8938738595688879, 'n': 4.340494882232946}
    social_params:  {'K': 0.8702480308329354, 'n': 1.9438551738813241}

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()